# Imports 

In [ ]:
import os

from langchain.tools import BaseTool, StructuredTool, tool
import pandas as pd

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "prj-ilios-ai.json"
os.environ["PROJECT_ID"] = "602280418311"
os.environ["LOCATION"] = "us-west1"
os.environ['GCS_BUCKET']="doc_ai_storage"

# Data loading

In [ ]:
site_lease = pd.read_excel("data/project_previews/site_lease_project_preview.xlsx")

In [ ]:
site_lease.head()

## Agents

In [ ]:
import enum
from enum import EnumMeta

site_lease_dict = {'_'.join(key.split(' ')).replace('(', '').replace(')', '').replace('/', ''): key for key in site_lease['Key Items'].values.tolist()}

class KeyItem(enum.Enum):
    __members__ = {name: value for name, value in site_lease_dict.items()}

In [ ]:
class KeyItem(enum.Enum):
    LESSOR_ENTITY_NAME = 'Lessor (Landlord) Entity Name'
    LESSEE_ENTITY_NAME = 'Lessee (Tenant) Entity Name'
    EFFECTIVE_DATE = 'Effective Date'
    PROPERTY_SIZE = 'Property Size'
    INITIAL_TERM = 'Initial Term'
    RENT_COMMENCEMENT = 'Rent Commencement'
    RENEWAL_TERMS = 'Renewal Terms'
    RENT_CALCULATION = 'Rent Calculation'


In [ ]:
{name: value for name, value in site_lease_dict.items()}

In [ ]:
[e.name for e in KeyItemLease]

In [ ]:
site_lease[site_lease['Key Items'] == KeyItemLease.LESSOR_ENTITY_NAME.value].loc[:, 'Legal Terms'].values

In [ ]:
key_item_search(KeyItemLease.LESSOR_ENTITY_NAME)

In [ ]:
# Get the prompt to use - you can modify this!
from langchain import hub
from langchain_google_vertexai import VertexAI
from langchain.agents import AgentExecutor, create_react_agent

# llm = VertexAI(  # type: ignore
#     model="text-bison-32k@002", project="prj-ilios-ai"
# )

@tool
def key_item_search(key_item: str) -> str:
    """You have a dictionary of terms extracted from the documents. You can search for a specific key and get the extracted answer"""
    fs = FuzzySet([e.value for e in KeyItem])
    best_choice = fs.get(key_item)
    if best_choice[0][0] < 0.8:
        return f"Could not find a match for '{key_item}'. Please use one of the following key items: {[e.value for e in KeyItem]}"
    return site_lease[site_lease['Key Items'] == best_choice[0][1]].loc[:, 'Legal Terms'].values[0]
    # if key_item not in KeyItem.__members__:
    #     return f"'{key_item}' is not a valid key item. Please use one of the following key items: {[e.value for e in KeyItem]}"
    # else:
    #     return site_lease[site_lease['Key Items'] == key_item].loc[:, 'Legal Terms'].values[0]

@tool
def cat_search(cat_name: str) -> str:
    """Returns information about where is the cat"""
    return f"{cat_name} is under the bed"

prompt = hub.pull("hwchase17/react")
tools = [key_item_search, cat_search]
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
key_item_search("Lessor (Landlord) Entity Name")

In [ ]:
[e.value for e in KeyItem]

In [ ]:
prompt

In [ ]:
llm = VertexAI(model="text-bison-32k@002", project="prj-ilios-ai")

In [ ]:
from langchain.agents import initialize_agent, AgentType

llm_chain = initialize_agent(
    tools,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    llm=llm,
    verbose=True,
)

In [ ]:
agent_executor.invoke({"input": "What is the landlord?"})

In [ ]:
from src.vectordb.gcp_vector_search.transform_and_load import single_text_embedding